# Risk Parity & SG Trend Follower Replication

In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt

# Get Data

In [2]:
def _get_raw_futures() -> pd.DataFrame: 

    data_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "data")
    futures_path = os.path.join(data_path, "futures.parquet")
    good_contracts = ["CL", "GC", "CO", "S", "ES", "VG", "Z", "NK", "TY", "G", "RX", "JGB"]

    df_tmp = (pd.read_parquet(
        path = futures_path, engine = "pyarrow").
        assign(
            sec_tmp = lambda x: x.security.str.split(" ").str[0],
            sec_name = lambda x: np.where(x.sec_tmp.str.len() > 1, x.sec_tmp.str[:-1], x.sec_tmp))
        [["date", "sec_name", "PX_LAST"]].
        rename(columns = {"sec_name": "security"}).
        query("security == @good_contracts"))
    
    return df_tmp

df_fut_raw = _get_raw_futures()

In [3]:
def _get_raw_fx() -> pd.DataFrame: 
    
    data_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "data")
    fx_path = os.path.join(data_path, "fx.parquet")

    df_tmp = (pd.read_parquet(
        path = fx_path, engine = "pyarrow")
        ["Adj Close"].
        reset_index().
        melt(id_vars = "Date").
        assign(variable = lambda x: x.variable.str.split("=").str[0]))
    
    return df_tmp

df_fx_raw = _get_raw_fx()

In [4]:
def prep_data() -> pd.DataFrame:
    
    df_fx_prep = (_get_raw_fx().rename(
        columns = {
            "variable": "security",
            "value": "px",
            "Date": "date"}))

    df_fut_prep = (_get_raw_futures().rename(
        columns = {
            "PX_LAST": "px"}))

    df_combined = pd.concat([df_fx_prep, df_fut_prep])
    return df_combined

df_combined = prep_data()

In [5]:
df_combined.security.drop_duplicates().sort_values().to_list()

['AUD', 'CHF', 'ES', 'EUR', 'GBP', 'JPY', 'NK', 'VG', 'Z']